In [1]:
from sagemaker.huggingface import HuggingFaceModel


/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
from sagemaker import get_execution_role
role = get_execution_role()

In [3]:
model_data = 's3://news-headlines-4cls/outputs/news-classifier-distilbert-2025-04-19-10-25-19-455/output/model.tar.gz'

In [4]:
# Create the model
huggingface_model = HuggingFaceModel(
    model_data=model_data,
    entry_point="inference2.py",
    source_dir=".",
    role=role,
    transformers_version="4.26",
    pytorch_version="1.13",
    py_version="py39"
)


In [5]:
# Deploy the endpoint
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name="news-distilbert-endpoint-v1"
)

[04/19/25 21:28:17] INFO     Repacking model artifact                                                  ]8;id=144011;file:///opt/conda/lib/python3.11/site-packages/sagemaker/model.py\model.py]8;;\:]8;id=465298;file:///opt/conda/lib/python3.11/site-packages/sagemaker/model.py#819\819]8;;\
                             (s3://news-headlines-4cls/outputs/news-classifier-distilbert-2025-04-19-1             
                             0-25-19-455/output/model.tar.gz), script artifact (.), and dependencies               
                             ([]) into single tar.gz file located at                                               
                             s3://sagemaker-us-east-1-658974490035/huggingface-pytorch-inference-2025-             
                             04-19-21-28-17-431/model.tar.gz. This may take some time depending on                 
                             model size...                                                                         

[04/19/25 21:30:53] INFO     Creating model with name:                                              ]8;id=140701;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=241726;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#4094\4094]8;;\
                             huggingface-pytorch-inference-2025-04-19-21-30-53-131                                 

                    INFO     Creating endpoint-config with name news-distilbert-endpoint-v1         ]8;id=522783;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=73321;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#5889\5889]8;;\

[04/19/25 21:30:54] INFO     Creating endpoint with name news-distilbert-endpoint-v1                ]8;id=113945;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=765414;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#4711\4711]8;;\

---------!

In [6]:
data = {
    "inputs": [
        "Google unveils new AI chip",
        "Fed raises interest rates again",
        "'I'm giving up': Cate Blanchett says she is retiring from acting",
        "Pandemic Exacerbates Inequities, Leaving Children with Lasting Psychological Effects"
    ]
}
response = predictor.predict(data)

for txt, cls, proba in zip(data['inputs'], response['label'], response['proba']):
    print(f'predicted "{cls}" for\ntxt: {txt}')
    print(f'probabilities => {proba}')
    print('=' * 27)

predicted "t" for
txt: Google unveils new AI chip
probabilities => {'t': 0.9994, 'b': 0.0004, 'm': 0.0002, 'e': 0.0}
predicted "b" for
txt: Fed raises interest rates again
probabilities => {'b': 0.9991, 'e': 0.0005, 'm': 0.0002, 't': 0.0002}
predicted "e" for
txt: 'I'm giving up': Cate Blanchett says she is retiring from acting
probabilities => {'e': 0.9998, 'b': 0.0001, 'm': 0.0001, 't': 0.0}
predicted "m" for
txt: Pandemic Exacerbates Inequities, Leaving Children with Lasting Psychological Effects
probabilities => {'m': 0.9996, 'b': 0.0002, 'e': 0.0001, 't': 0.0001}


In [7]:
# Clean up the resources
predictor.delete_endpoint()


[04/19/25 21:35:56] INFO     Deleting endpoint configuration with name: news-distilbert-endpoint-v1 ]8;id=103384;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=647054;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#4865\4865]8;;\

                    INFO     Deleting endpoint with name: news-distilbert-endpoint-v1               ]8;id=728663;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=584865;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#4855\4855]8;;\